In [1]:
import pandas as pd
import numpy as np
import re

#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, \
                            classification_report, precision_recall_curve, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

import dill

In [2]:
def clean_msg(msg):
    if isinstance(msg, list):
        list_len = len(msg) - 2
        for pos, elem in enumerate(msg):
            if isinstance(elem, dict) and pos ==  list_len:
                for key, val in elem.items():
                    if key == 'type' and val == 'hashtag':
                        break
                    if key == 'type' and val == 'bold':
                        break
                    if key == 'type' and val == 'mention_name':
                        break
                    if key == 'text':
                        return val
    if isinstance(msg, str):
        return msg


In [3]:
ham_raw = pd.read_json('../../data/spam_detection/ham2016.json')
spam_raw = pd.read_json('../../data/spam_detection/spam.json')

## Обработка неспамных сообщений

In [4]:
ham_raw.head(3)

,name,type,id,messages
0,Новое Измайлово,public_supergroup,9666949336,"{'id': -999996617, 'type': 'service', 'date': ..."
1,Новое Измайлово,public_supergroup,9666949336,"{'id': 1, 'type': 'service', 'date': '2016-08-..."
2,Новое Измайлово,public_supergroup,9666949336,"{'id': 99, 'type': 'service', 'date': '2016-09..."


In [5]:
ham_raw1 = pd.json_normalize(ham_raw['messages'])
pd.set_option('display.max_columns', 50)
ham_raw1.tail(3)

,id,type,date,actor,actor_id,action,text,title,members,from,from_id,file,thumbnail,media_type,sticker_emoji,width,height,reply_to_message_id,photo,edited,forwarded_from,mime_type,contact_information.first_name,contact_information.last_name,contact_information.phone_number,duration_seconds,via_bot,contact_vcard,inviter,message_id,poll.question,poll.closed,poll.total_voters,poll.answers,location_information.latitude,location_information.longitude
60987,73759,message,2021-03-24T09:52:51,NaN,NaN,NaN,мойка проходит в несколько этапов: сначала сби...,NaN,NaN,Серёжа,4.801717e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60988,73764,message,2021-03-24T10:08:35,NaN,NaN,NaN,Спасибо большое! Попробую сегодня,NaN,NaN,ИриШок,5.225578e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60989,73765,message,2021-03-24T10:08:35,NaN,NaN,NaN,"[{'type': 'mention_name', 'text': 'Серёжа', 'u...",NaN,NaN,ChatKeeperBot,4.848115e+09,NaN,NaN,NaN,NaN,NaN,NaN,73759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Пример предпоследнего сообщения

In [6]:
ham_raw1['text'][60988]

'Спасибо большое! Попробую сегодня'

Пример последнего сообщения от бота

In [7]:
ham_raw1['text'][60989]

[{'type': 'mention_name', 'text': 'Серёжа', 'user_id': 506749612},
 ', вашу репутацию увеличил ',
 {'type': 'mention_name', 'text': 'ИриШок', 'user_id': 930610950},
 '.\n Ваша репутация 8']

Список признаков

In [8]:
ham_raw1.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude'],
      dtype='object')

Уникальные значения по некоторым признакам

In [9]:
all_cols = ['id', 'type', 'date', 'actor', 'actor_id', 'action', 'text', 'title',
       'members', 'from', 'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'width', 'height', 'reply_to_message_id', 'photo',
       'edited', 'forwarded_from', 'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
       'contact_vcard', 'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 'poll.answers',
       'location_information.latitude', 'location_information.longitude']

some_cols = ['id', 'type', 'date', 'actor_id', 'action',  
         'from_id', 'file', 'thumbnail', 'media_type',
       'sticker_emoji', 'reply_to_message_id', 'photo',
       'edited',  'mime_type',
       'contact_information.first_name', 'contact_information.last_name',
       'contact_information.phone_number', 'duration_seconds', 'via_bot',
        'inviter', 'message_id', 'poll.question',
       'poll.closed', 'poll.total_voters', 
       'location_information.latitude', 'location_information.longitude']

for col in some_cols:
    print(col, ham_raw1[col].unique())

id [-999996617          1         99 ...      73759      73764      73765]
type ['service' 'message']
date ['2016-08-26T12:01:01' '2016-09-01T18:04:20' '2016-09-01T18:07:07' ...
 '2021-03-24T09:48:16' '2021-03-24T09:52:51' '2021-03-24T10:08:35']
actor_id [4.49716059e+09 9.66694934e+09 4.48584734e+09 ... 4.86776919e+09
 5.43563062e+09 5.47227313e+09]
action ['migrate_to_supergroup' 'migrate_from_group' 'invite_members' nan
 'remove_members' 'join_group_by_link' 'pin_message' 'group_call'
 'invite_to_group_call']
from_id [           nan 4.47956145e+09 4.55065251e+09 ... 5.37073416e+09
 5.95687004e+09 5.75695058e+09]
file [nan '(File not included. Change data exporting settings to download.)']
thumbnail [nan '(File not included. Change data exporting settings to download.)']
media_type [nan 'sticker' 'video_file' 'animation' 'voice_message' 'video_message'
 'audio_file']
sticker_emoji [nan '😠' '💩' '🎉' '🌚' '😊' '🙈' '😒' '🎮' '😂' '🙄' '👈' '😳' '🤔' '😌' '😞' '😋' '😍'
 '☺️' '😁' '👍' '✌' '💐' '😡' '🔥' '😐

In [10]:
ham_raw1['text'].tail(10)

60980    [{'type': 'mention_name', 'text': 'بلا أم أحمق...
60981    Здравствуйте! Скажите пожалуйста ,а машина сух...
60982           Здравствуйте, да, там сушка автоматическая
60983                       На объездном  сейчас очередь (
60984                              И не царапает покрытие?
60985                              Присоединяюсь к вопросу
60986    лично у меня ни разу не царапало.. но я езжу т...
60987    мойка проходит в несколько этапов: сначала сби...
60988                    Спасибо большое! Попробую сегодня
60989    [{'type': 'mention_name', 'text': 'Серёжа', 'u...
Name: text, dtype: object

In [11]:
# извлечение из json текста, , замена '' на Nan, удаление строк (сообщений) без текста
# обновление индекса после удаление строк (индекс теперь по порядку)
ham = ham_raw1['text'].apply(clean_msg).replace('', np.nan).dropna().reset_index(drop=True)
ham

0                                                  Всем ✋🏻
1        Это супергруппа. А это значит что админ может ...
2                                                        /
3                    не пугайтесь, немного почистил мусор)
4                                             Всем привет)
                               ...                        
54382                              И не царапает покрытие?
54383                              Присоединяюсь к вопросу
54384    лично у меня ни разу не царапало.. но я езжу т...
54385    мойка проходит в несколько этапов: сначала сби...
54386                    Спасибо большое! Попробую сегодня
Name: text, Length: 54387, dtype: object

In [12]:
# увеличение максимальной ширины столбца
pd.set_option('max_colwidth', 110)
ham.head(50)

0                                                                                                           Всем ✋🏻
1     Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...
2                                                                                                                 /
3                                                                             не пугайтесь, немного почистил мусор)
4                                                                                                      Всем привет)
5                                                                                                       Привет всем
6                                                                                                      Добрый вечер
7                                                                                                   Доброе утро! :)
8     Дети разные бывают - если это маленькие - то родителям обьяснил бы

## Обработка сообщений со спамом

In [13]:
spam_raw.head(3)

,name,type,id,messages
0,Журнал НИ,private_channel,9802778229,"{'id': 1, 'type': 'service', 'date': '2020-07-07T20:25:57', 'actor': 'Журнал НИ', 'actor_id': 9802778229, ..."
1,Журнал НИ,private_channel,9802778229,"{'id': 4, 'type': 'message', 'date': '2020-07-07T20:27:15', 'from': 'Журнал НИ', 'from_id': 9802778229, 't..."
2,Журнал НИ,private_channel,9802778229,"{'id': 5, 'type': 'message', 'date': '2020-07-07T20:31:15', 'from': 'Журнал НИ', 'from_id': 9802778229, 't..."


In [14]:
spam_raw2 = pd.json_normalize(spam_raw['messages'])
pd.set_option('display.max_columns', 50)
spam_raw2.tail(10)

,id,type,date,actor,actor_id,action,title,text,from,from_id
2003,2044,message,2021-03-23T13:22:14,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n, {'type': 'code', 'text': ...",Журнал НИ,9.802778e+09
2004,2045,message,2021-03-23T15:26:59,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой mute\n17aa7c06-b7bb-4ecc-a0f2-9beda0594b9c\n\n, {'type': 'code', 'text': ...",Журнал НИ,9.802778e+09
2005,2046,message,2021-03-23T21:30:20,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, , {'type': 'hashtag', 'text': '#новыйпользователь...",Журнал НИ,9.802778e+09
2006,2047,message,2021-03-23T21:34:33,NaN,NaN,NaN,NaN,"[⚙️ , {'type': 'hashtag', 'text': '#Настройки'}, Пользователь изменил настройки группы , {'type': 'hashta...",Журнал НИ,9.802778e+09
2007,2048,message,2021-03-24T09:12:42,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, , {'type': 'hashtag', 'text': '#новыйпользователь...",Журнал НИ,9.802778e+09
2008,2049,message,2021-03-24T09:13:01,NaN,NaN,NaN,NaN,"[✅ Пользователь прошедший проверку разблокирован , {'type': 'hashtag', 'text': '#запрос'}, \n133e204a-8a1b...",Журнал НИ,9.802778e+09
2009,2050,message,2021-03-24T09:14:22,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n6ae21a32-1bdf-47c5-a940-f2b13181e8aa\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2010,2051,message,2021-03-24T09:14:32,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n63942221-6053-4e57-af28-b0b54d7f7bc1\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2011,2052,message,2021-03-24T09:14:53,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n917036a2-8087-4607-94fa-60b632b866f7\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09
2012,2053,message,2021-03-24T09:18:28,NaN,NaN,NaN,NaN,"[🆘 Пользователь наказан командой ban\n4502aa45-f47b-4415-891b-5baf3368fde1\n\n, {'type': 'code', 'text': '...",Журнал НИ,9.802778e+09


In [15]:
spam_raw2.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'title', 'text',
       'from', 'from_id'],
      dtype='object')

In [16]:
for col in ['type', 'actor', 'actor_id', 'action', 'title', 'from', 'from_id']:
    print(col, spam_raw2[col].unique())

type ['service' 'message']
actor ['Журнал НИ' nan]
actor_id [9.80277823e+09            nan]
action ['create_channel' nan]
title ['Журнал НИ' nan]
from [nan 'Журнал НИ']
from_id [           nan 9.80277823e+09]


In [17]:
spam_raw3 = spam_raw2['text']
spam_raw3.tail(15)

1998    [🆘 Пользователь наказан командой mute\nd5bce06f-b34c-4994-8aea-0692697036d2\n\n, {'type': 'code', 'text': ...
1999    [🆘 Пользователь наказан командой mute\n3e7a9f94-d819-4e23-9174-f78cf59990e0\n\n, {'type': 'code', 'text': ...
2000    [🆘 Пользователь наказан командой mute\n577e6a18-48bb-4c13-9c89-a7c240574859\n\n, {'type': 'code', 'text': ...
2001    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'},  , {'type': 'hashtag', 'text': '#новыйпользователь...
2002    [✅ Пользователь прошедший проверку разблокирован , {'type': 'hashtag', 'text': '#запрос'}, \n09698fee-2773...
2003    [🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n, {'type': 'code', 'text': ...
2004    [🆘 Пользователь наказан командой mute\n17aa7c06-b7bb-4ecc-a0f2-9beda0594b9c\n\n, {'type': 'code', 'text': ...
2005    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'},  , {'type': 'hashtag', 'text': '#новыйпользователь...
2006    [⚙️ , {'type': 'hashtag', 'text': '#Настройки'},

In [18]:
spam_raw3.iloc[2003]

['🆘 Пользователь наказан командой mute\n1c565fd4-fff4-4d57-a7e1-e88444b10d3a\n\n',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 '] ',
 {'type': 'italic', 'text': 'ограничил возможность писать сообщения'},
 ' ',
 {'type': 'code', 'text': 'МГТС'},
 ' [',
 {'type': 'mention_name', 'text': '1176395792', 'user_id': 1176395792},
 '] [',
 {'type': 'hashtag', 'text': '#user1176395792'},
 '] на 24 часов\n\n\n\n',
 {'type': 'bold', 'text': 'Группа:'},
 ' ',
 {'type': 'code', 'text': 'Новое Измайлово'},
 ' [',
 {'type': 'mention', 'text': '@new_izmailovo'},
 '] [',
 {'type': 'hashtag', 'text': '#chat1001077014744'},
 ']\n',
 {'type': 'bold', 'text': 'Инициатор:'},
 ' ',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 ']\n',
 {'type': 'bold', 'text': 'Пользователь:'},
 ' ',
 {'type': 'code', 

In [19]:
clean_msg(spam_raw3.iloc[2003])

'Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон: +7(499)2380071'

In [20]:
spam = spam_raw3.apply(clean_msg)
spam.tail(10)

2003    Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...
2004    Здравствуйте! Услуги Психолога!\n\nПсихолог с высшим образованием и практикой более 10 лет предлагаю услуг...
2005                                                                                                             None
2006                                                                                                             None
2007                                                                                                             None
2008                                                                                                             None
2009                                                                                                             None
2010                                                                                                             None
2011                                                    

In [21]:
# извлечение из json текста, удаление строк (сообщений) без текста,
# обновление индекса после удаление строк (индекс теперь по порядку)
spam = spam_raw3.apply(clean_msg).dropna().reset_index(drop=True)

In [22]:
spam

0                                                                                                                   
1                                                                             Канал успешно добавлен в журнал группы
2         Добрый вечер!Предлагаю услуги няни.Свободно до 1 сентября.без вредных привычек.Опыт большой  ,люблю детей.
3      Приветствую 🙂\nКУПЛЮ ОДНОКОМНАТНУЮ (от 38м) квартиру в Новом Измайлово\nс ремонтом, мебелью, бытовой техни...
4                                                  Черничка\nДоставка черники по Балашихе\nhttps://t.me/chernichka03
                                                           ...                                                      
182    Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...
183                                                                      https://t.me/MoscowskyPravdorub/1470?single
184                                                             

## Получение объединённого датасета

In [23]:
# Series -> DataFrame
ham_df = ham.to_frame('msg')
# создание нового столбца с категорией спам / не спам
ham_df['spam'] = 0

spam_df = spam.to_frame('msg')
spam_df['spam'] = 1

# конкатенация датафреймов
df = pd.concat([ham_df, spam_df], axis=0, ignore_index=True)
df

,msg,spam
0,Всем ✋🏻,0
1,Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...,0
2,/,0
3,"не пугайтесь, немного почистил мусор)",0
4,Всем привет),0
...,...,...
54569,"Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...",1
54570,https://t.me/MoscowskyPravdorub/1470?single,1
54571,Большой выбор ждём вас,1
54572,Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...,1


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('spam', 1), 
                                                    df['spam'], random_state=0)
#save test
X_test.to_csv("X_test_spam.csv", index=None)
y_test.to_csv("y_test_spam.csv", index=None)
#save train
X_train.to_csv("X_train_spam.csv", index=None)
y_train.to_csv("y_train_spam.csv", index=None)

In [25]:
X_train

,msg
3856,"Они в один месяц не учли показания (программа сбойнула, не загрузилось с сайта, не туда посмотрели) и у ва..."
40894,"Раз в час примерно, первая в районе 7 утра уходит."
7915,Это же гениально! 😂отличная идея!)))
25371,"Да непонятно, в доброделе отвечают — проведите ОСС и сами залейте)"
40093,"Может одна новая, пробега мало, а вторая загаженная"
...,...
45891,Спасибо 😀
52416,А разве заставляют?
42613,"Не то футбольный праздник, не то свадьба в Чечне."
43567,А чат собаководов есть?


In [26]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [27]:
features = ['msg']
target = 'spam'

In [28]:
#combine
msg = Pipeline([
                ('imputer', TextImputer('msg', '')),
                ('selector', ColumnSelector(key='msg')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('msg', msg)])

In [29]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Wall time: 723 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('msg',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='msg',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='msg')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10))]))])),
                ('classifier', LogisticRegression())])

Посмотрим, как выглядит наш pipeline

In [30]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('msg',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='msg',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='msg')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))]))])),
 ('classifier', LogisticRegression())]

In [31]:
predictions = pipeline.predict_proba(X_test)
#pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [32]:
y_test

48502    0
48785    0
40039    0
40399    0
50011    0
        ..
27058    0
16446    0
42301    0
11306    0
31998    0
Name: spam, Length: 13644, dtype: int64

In [33]:
predictions[:, 1][:]

array([0.00166743, 0.00600112, 0.00191033, ..., 0.0022781 , 0.00163441,
       0.00238243])

In [34]:
roc_auc_score(y_score=predictions[:, 1], y_true=y_test)

0.9552632018716577

In [35]:
f1_score(predictions[:, 1].round(), y_test)

0.0

In [36]:
precision_score(predictions[:, 1].round(), y_test)

0.0

In [37]:
recall_score(predictions[:, 1].round(), y_test)

C:\Users\Alex\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [38]:
df_pred = pd.concat([df, pd.Series(predictions[:, 1])], axis=1)
df_pred.columns=["msg", "spam", "prob"]
df_pred

,msg,spam,prob
0,Всем ✋🏻,0,0.001667
1,Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...,0,0.006001
2,/,0,0.001910
3,"не пугайтесь, немного почистил мусор)",0,0.001192
4,Всем привет),0,0.004088
...,...,...,...
54569,"Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...",1,NaN
54570,https://t.me/MoscowskyPravdorub/1470?single,1,NaN
54571,Большой выбор ждём вас,1,NaN
54572,Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...,1,NaN


In [39]:
df_pred.iloc[54570,2], type(df_pred.iloc[54570,2])

(nan, numpy.float64)

In [40]:
pd.set_option('display.max_rows', 200)
df_pred.loc[df_pred['prob'] > 0.1]

,msg,spam,prob
3458,Брат окно,0,0.165447
4849,Воробей всегда подрузамевает))))))))),0,0.143628
6068,Они по проекту района там предполагались,0,0.161846
7398,Желания нет продолжать эту беседу...,0,0.161846
8806,https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku,0,0.161846
9038,Включенные габариты,0,0.161846
9071,"Да, смартфон Android, компьютер Windows",0,0.117427
9257,"Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...",0,0.161846
9632,"И кстати вот тут не за что извиняться мы ж русские , и ""вы"" на Руси было оскорбительно",0,0.166958
9922,ЗАметано),0,0.111680


In [41]:
pd.set_option('display.max_rows', 200)
df_pred.query('(prob > 0.15)').head(200)

,msg,spam,prob
3458,Брат окно,0,0.165447
6068,Они по проекту района там предполагались,0,0.161846
7398,Желания нет продолжать эту беседу...,0,0.161846
8806,https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku,0,0.161846
9038,Включенные габариты,0,0.161846
9257,"Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...",0,0.161846
9632,"И кстати вот тут не за что извиняться мы ж русские , и ""вы"" на Руси было оскорбительно",0,0.166958
10149,"Ну и ищите, пойду инвесторов в дивизию предлагать",0,0.161846
10625,"Если брать за понятие ""живёт"" - ставит себе на стол тарелку с супом, то да ЖИВЁТ. А если брать возможности...",0,0.179452
11090,"жена тоже рыба блять, молчит в тряпочку",0,0.161846


Сохраним модель (пайплайн)

In [42]:
with open("spam_classifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)